In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3768f87fd93c41dfaebfe9b09a017be2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='n2XJsCIXYzOUkku7S_dmtNq5DEeAWWXWEzBTUVKYfccp',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_3768f87fd93c41dfaebfe9b09a017be2.get_object(Bucket='cnnecgsowjanya-donotdelete-pr-0t1mjcp3vfw2qg', Key='data.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [13]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

# ECG-Arrhythmia Image Classification

### Image Processing

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [20]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

### Model Building

In [141]:
model = Sequential()

##### Adding CNN Layers

In [142]:
model.add(Convolution2D(32,(3,3),input_shape = (64,64,3),activation = "relu"))

In [143]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [144]:
model.add(Convolution2D(32,(3,3),activation='relu'))

In [145]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [146]:
model.add(Flatten()) # input of ANN

##### Adding Dense Layers

In [147]:
model.add(Dense(units = 128,kernel_initializer = "random_uniform",activation = "relu"))

In [148]:
model.add(Dense(units = 128,kernel_initializer = "random_uniform",activation = "relu"))

In [151]:
model.add(Dense(units = 128,kernel_initializer = "random_uniform",activation = "relu"))

In [152]:
model.add(Dense(units = 128,kernel_initializer = "random_uniform",activation = "relu"))

##### Output Layer

In [153]:
model.add(Dense(units = 6,kernel_initializer = "random_uniform",activation = "softmax"))

In [154]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6272)              0         
                                                                 
 dense_10 (Dense)            (None, 128)              

##### Configuring learning process

In [155]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [156]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10
480/480 [==============================] - 56s 115ms/step - loss: 1.3865 - accuracy: 0.4952 - val_loss: 1.6698 - val_accuracy: 0.4758
Epoch 2/10
480/480 [==============================] - 55s 114ms/step - loss: 0.7554 - accuracy: 0.7317 - val_loss: 1.0585 - val_accuracy: 0.6860
Epoch 3/10
480/480 [==============================] - 55s 115ms/step - loss: 0.3591 - accuracy: 0.8767 - val_loss: 0.6107 - val_accuracy: 0.7982
Epoch 4/10
480/480 [==============================] - 56s 118ms/step - loss: 0.2319 - accuracy: 0.9293 - val_loss: 0.5173 - val_accuracy: 0.8372
Epoch 5/10
480/480 [==============================] - 56s 117ms/step - loss: 0.1676 - accuracy: 0.9483 - val_loss: 0.4504 - val_accuracy: 0.8558
Epoch 6/10
480/480 [==============================] - 57s 118ms/step - loss: 0.1264 - accuracy: 0.9626 - val_loss: 0.4050 - val_accuracy: 0.8768
Epoch 7/10
480/480 [==============================] - 56s 116ms/step - loss: 0.1084 - accuracy: 0.9659 - val_loss: 0.5415 - val_ac

In [157]:
model.save('ECG_IBM.h5')

In [158]:
!tar -czvf ECG-arrhythmia-classification-model_new.tgz ECG_IBM.h5

ECG_IBM.h5


In [159]:
ls -1

data/
ECG-arrhythmia-classification-model_new.tgz
ECG.h5
ECG_IBM.h5
my_model.tar.gz


In [160]:
!pip install watson-machine-learning-client --upgrade

In [161]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"3oTF4pfdIUJbXlIqIs4agFrNs5CWJKADACv5uoqof-qj"
}
client = APIClient(wml_credentials=wml_credentials)

In [162]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------------  ------------------------
ID                                    NAME                CREATED
2a2ff914-6b9d-492d-b474-826e17626e1b  ECG_Classification  2022-07-05T10:53:42.674Z
------------------------------------  ------------------  ------------------------


In [163]:
space_uid = "2a2ff914-6b9d-492d-b474-826e17626e1b"

In [164]:
space_uid=guid_from_space_name(client,'ECG_Classification')
print("Space UID= "+space_uid)

Space UID= 2a2ff914-6b9d-492d-b474-826e17626e1b


In [165]:
client.set.default_space(space_uid)

'SUCCESS'

In [166]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [167]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [168]:
model_details = client.repository.store_model(
    model='ECG-arrhythmia-classification-model_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
    }
)
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [169]:
model_id

'17f9a6f1-b70f-49c2-8959-142146799501'

In [ ]:
client.repository.download(model_id,'my_model.tar.gz')

### Test the Model

In [172]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model=load_model("/home/wsuser/work/data/ECG_IBM.h5")
img=image.load_img("/home/wsuser/work/testing.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = model.predict(x)
y_pred=np.argmax(pred)
y_pred

index=['left Bundle Branch block',
       'Normal',
       'Premature Atrial Contraction',
       'Premature Ventricular Contraction',
       'Right Bundle Branch Block',
       'Ventricular Fibrillation']
result = str(index[y_pred])
result

OSError: No file or directory found at /home/wsuser/work/data/ECG_IBM.h5